# Simulate Buy and Hold Strategy, More Detailed Monte Carlo Analysis & Test Transaction Cost Model 

In [ ]:
import pandas as pd
from pathlib import Path
from dateutil.relativedelta import relativedelta

In [ ]:
from utils.math import perform_monte_carlo_simulation
from utils.portfolio import GermanTaxModel, MAPortfolio, Portfolio

In [ ]:
# load raw and clean data
clean_data_path = Path("clean_data")
cache_path = Path("cached_data")

In [ ]:
# get preprocessed asset data and convert to have useable index
input_path = clean_data_path / "etfs.xlsx"
etfs = pd.read_excel(input_path, index_col=0)
etfs.index = pd.to_datetime(etfs.index)
etfs['cash'] = 100.0
etfs.head()

In [ ]:
# Volatilty tests
from utils.plots import draw_growth_chart
from utils.math import calc_volatility_over_time

data = etfs.pct_change()

volatility_weekly = calc_volatility_over_time(data, average_size=relativedelta(days = 5), step_size=relativedelta(days=1), assets=["1x_sp500_eu"])
volatility_monthly = calc_volatility_over_time(data, average_size=relativedelta(months = 1), step_size=relativedelta(days=1), assets=["1x_sp500_eu"])
volatility_yearly = calc_volatility_over_time(data, average_size=relativedelta(years = 1), step_size=relativedelta(days=1), assets=["1x_sp500_eu"])

draw_growth_chart(
    {
        "S&P 500 EU": data['1x_sp500_eu'],
        "Weekly Averaged Volatility": volatility_weekly["1x_sp500_eu"],
        "Monthly Averaged Volatility": volatility_monthly["1x_sp500_eu"],
        "Yearly Averaged Volatility": volatility_yearly["1x_sp500_eu"],
    },
    "Yields over Time",
    y_log = False,
    y_title = "yields in %",
)

In [ ]:
from utils.math import calc_growth

draw_growth_chart(
    {
        "S&P 500 EU": calc_growth(data['1x_sp500_eu'].iloc[1:]),
        "Weekly Averaged Volatility": volatility_weekly["1x_sp500_eu"],
        "Monthly Averaged Volatility": volatility_monthly["1x_sp500_eu"],
        "Yearly Averaged Volatility": volatility_yearly["1x_sp500_eu"],
    },
    "Yields over Time",
    y_log = False,
    y_title = "yields in %",
)

In [ ]:
from utils.portfolio import VolPortfolio

p_cash = MAPortfolio(
    {
        'cash': dict(dist=100),
    },
    start_value = 10000,
).backtest(etfs)

p_2x_sp500_vol = VolPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, vol=30, vol_2=5, vol_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

p_2x_sp500_ma = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=290, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

# Add buy and hold of 2x S&P 500
p_2x_sp500 = Portfolio(
    {
        '2x_sp500_eu': 100.0,
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

In [ ]:
# define simulation period properties
start_value = 10000
monthly_rate = 100
simulation_time = relativedelta(years = 15)

In [ ]:
# Add 2x S&P 500 volatility (with and without monthly rate)
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_vol,
    portfolio_name = '2x S&P 500 Volatility',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_vol,
    portfolio_name = '2x S&P 500 Volatility',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 Moving Average',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 Moving Average',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

In [ ]:
p_cash = MAPortfolio(
    {
        'cash': dict(dist=100),
    },
    start_value = 10000,
).backtest(etfs)

p_2x_sp500_ma200 = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=200, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

In [ ]:
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma200,
    portfolio_name = '2x S&P 500 Moving Average (200 d)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma200,
    portfolio_name = '2x S&P 500 Moving Average (200 d)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

### Define Portfolios to Test

With the strategy also used in notebook 13. However, add here the 2x leveraged S&P 500 buy and hold strategy.

Additionally, play around with the new growth histogram, amount of Monte Carlo simulations to check for convergence and with different time periods.  

In [ ]:
p_cash = MAPortfolio(
    {
        'cash': dict(dist=100),
    },
    start_value = 10000,
).backtest(etfs)

p_2x_sp500_ma = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=290, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

# Add buy and hold of 2x S&P 500
p_2x_sp500 = Portfolio(
    {
        '2x_sp500_eu': 100.0,
    },
    start_value = 10000,
    tax_model=GermanTaxModel(),
).backtest(etfs)

In [ ]:
# define simulation period properties
start_value = 10000
monthly_rate = 100
simulation_time = relativedelta(years = 15)

In [ ]:
# Add 2x S&P 500 buy and hold (with and without monthly rate)
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

fig1 = perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

fig2 = perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 100,
    reference_simulations = 1,
    seed=42,
)

# save as html to keep it interactive
fig1.write_html("images/2xBuyandHoldwithMonthly.html")
fig2.write_html("images/2xMAwithMonthly.html")

While the maximum as well as median and mean returns of the Moving Average strategy (290 day) are lower than of the Buy and Hold strategy, this mostly comes from the skewed return/ growth distribution. In terms of above's numbers, it means that for the Buy and Hold it is most likely that you end up at +/- 20% after 15 years, while the minimum return for the Moving Average is ~20%.

Now on to investigating the amount of samples we have to draw to get reliable statistics.

In [ ]:
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 500,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 1000,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

fig3 = perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 2000,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

# save as html to keep it interactive
fig3.write_html("images/2xMAwithMonthly_2000samples.html")

From brief experimentation, mean, median and quantiles only change slightly above 100 samples, likely also due to the large overlaps due to "only" 80 years of date for 15 year periods.

Now we can thus investigate this strategy for different investment horizons, assuming equal amount of samples required and taking 500 samples for a good histogram while still keeping some performance.

In [ ]:
for investment_horizon in [relativedelta(years = 5), relativedelta(years = 10), relativedelta(years = 20), relativedelta(years = 30), relativedelta(years = 50)]:
    # draw histogram
    perform_monte_carlo_simulation(
        portfolio = p_2x_sp500_ma,
        portfolio_name = '2x S&P 500 (MA)',
        reference = p_cash,
        reference_name = 'cash',
        simulation_time = investment_horizon,
        start_value = start_value,
        monthly_rate = 500,
        portfolio_simulations = 500,
        reference_simulations = 1,
        draw_hist=True,
        seed=42,
    )

    # make logarithmic plot
    perform_monte_carlo_simulation(
        portfolio = p_2x_sp500_ma,
        portfolio_name = '2x S&P 500 (MA)',
        reference = p_cash,
        reference_name = 'cash',
        simulation_time = investment_horizon,
        start_value = start_value,
        monthly_rate = 500,
        portfolio_simulations = 500,
        reference_simulations = 1,
        seed=42,
    )
    

In [ ]:
for investment_horizon in [relativedelta(years = 5), relativedelta(years = 10), relativedelta(years = 20), relativedelta(years = 30), relativedelta(years = 50)]:
    # draw histogram
    perform_monte_carlo_simulation(
        portfolio = p_2x_sp500,
        portfolio_name = '2x S&P 500 Buy and Hold',
        reference = p_cash,
        reference_name = 'cash',
        simulation_time = investment_horizon,
        start_value = start_value,
        monthly_rate = 500,
        portfolio_simulations = 500,
        reference_simulations = 1,
        draw_hist=True,
        seed=42,
    )

    # make logarithmic plot
    perform_monte_carlo_simulation(
        portfolio = p_2x_sp500,
        portfolio_name = '2x S&P 500 Buy and Hold',
        reference = p_cash,
        reference_name = 'cash',
        simulation_time = investment_horizon,
        start_value = start_value,
        monthly_rate = 500,
        portfolio_simulations = 500,
        reference_simulations = 1,
        seed=42,
    )

### As an Addition, Check Out the Effect of Earlier Buying Strategies (two MAs) and Transaction Costs 

But at first to the transactions costs, which have been added as optional to `portfolio` and `ma_portfolio`. 

In [ ]:
# define portfolios with transaction costs
p_2x_sp500_ma_cost = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=290, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    transaction_cost={'absolute': 4.9, 'relative': 0.0025, 'limit': 70},
    tax_model=GermanTaxModel(),
).backtest(etfs)

p_2x_sp500_ma_rel_cost = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=290, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    transaction_cost={'relative': 0.0025, 'limit': 70},
    tax_model=GermanTaxModel(),
).backtest(etfs)

# Add buy and hold of 2x S&P 500
p_2x_sp500_cost = Portfolio(
    {
        '2x_sp500_eu': 100.0,
    },
    start_value = 10000,
    transaction_cost={'absolute': 4.9, 'relative': 0.0025, 'limit': 70},
    tax_model=GermanTaxModel(),
).backtest(etfs)

In [ ]:
# compare both strategies with and without the transaction costs
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_cost,
    portfolio_name = '2x S&P 500 Buy and Hold with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,    
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma_cost,
    portfolio_name = '2x S&P 500 (MA) with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

# only relative  costs (like spread)
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma_rel_cost,
    portfolio_name = '2x S&P 500 (MA) with rel. Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)


As it should be, the Buy and Hold strategy stays unaffected. As we can see from comparing the last three graphs, mostly the relative transaction costs (assuming given absolute and relative transaction costs) eat up some portion of the larger interest Series, while the minimal growth is hardly affected. Mean is affected by -1%, median by -0.5% through costs.

With the costs included, we can finally test the earlier buying with a shorter moving average.

In [ ]:
p_2x_sp500_ma2_cost = MAPortfolio(
    {
        "2x_sp500_eu": dict(dist=100, ma=290, ma_2=60, ma_asset="1x_sp500_eu"),
    },
    start_value = 10000,
    transaction_cost={'relative': 0.0025, 'limit': 70},
    tax_model=GermanTaxModel(),
).backtest(etfs)

In [ ]:
# compare against previous MA strategies with and without costs
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma,
    portfolio_name = '2x S&P 500 (MA)',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma_cost,
    portfolio_name = '2x S&P 500 (MA) with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma2_cost,
    portfolio_name = '2x S&P 500 (2x MA) with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 500,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)


One more thing to simulate is the cost-average effect: Is buying in tranches sensible?

In [ ]:
perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma_cost,
    portfolio_name = '2x S&P 500 (MA) with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500_ma_cost,
    portfolio_name = '2x S&P 500 (MA) with Costs',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = 0,
    monthly_rate = 2000,
    portfolio_simulations = 250,
    reference_simulations = 1,
    max_executions = 5,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = start_value,
    monthly_rate = 0,
    portfolio_simulations = 250,
    reference_simulations = 1,
    draw_hist=True,
    seed=42,
)

perform_monte_carlo_simulation(
    portfolio = p_2x_sp500,
    portfolio_name = '2x S&P 500 Buy and Hold',
    reference = p_cash,
    reference_name = 'cash',
    simulation_time = simulation_time,
    start_value = 0,
    monthly_rate = 2000,
    portfolio_simulations = 250,
    reference_simulations = 1,
    max_executions = 5,
    draw_hist=True,
    seed=42,
)